In [ ]:
#reference: https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv('drive/My Drive/Colab Notebooks/test.csv')
df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
df = df[~df.article.duplicated()]
df.loc[:,'article_length'] = df.article.apply(lambda x:len(x))


In [ ]:
import re
from nltk import tokenize
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
download('punkt')
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stop_words = stopwords.words('english')
articles = df['article'].tolist()


In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

ans = []
for article in tqdm(articles):
    cleaned = []
    summarize_text = ''
    article = tokenize.sent_tokenize(article)

    for sentence in article:
        sentence = re.sub(r'[^a-zA-Z0-9]', ' ', sentence)
        cleaned.append(sentence)

    similarity_matrix = np.zeros((len(cleaned), len(cleaned)))
    for i in range(len(cleaned)):
        for j in range(len(cleaned)):
            if i == j: 
                continue 
            similarity_matrix[i][j] = sentence_similarity(cleaned[i], cleaned[j], stop_words)

    sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(cleaned)), reverse=True)    
    for i in range(2):
        summarize_text += ranked_sentence[i][1]

    ans.append(summarize_text)
    
df['pred'] = ans
df.to_csv('drive/My Drive/Colab Notebooks/cosine.csv')

100%|██████████| 11488/11488 [2:33:51<00:00,  1.24it/s]
